# Road Following 

If you've run through the collision avoidance sample, your should be familiar following three steps

1.  Data collection
2.  Training
3.  Deployment

In this notebook, we'll do the same exact thing!  Except, instead of classification, you'll learn a different fundamental technique, **regression**, that we'll use to
enable JetBot to follow a road (or really, any path or target point).  

1. Place the JetBot in different positions on a path (offset from center, different angles, etc)

>  Remember from collision avoidance, data variation is key!

2. Display the live camera feed from the robot
3. Using a gamepad controller, place a 'green dot', which corresponds to the target direction we want the robot to travel, on the image.
4. Store the X, Y values of this green dot along with the image from the robot's camera

Then, in the training notebook, we'll train a neural network to predict the X, Y values of our label.  In the live demo, we'll use
the predicted X, Y values to compute an approximate steering value (it's not 'exactly' an angle, as
that would require image calibration, but it's roughly proportional to the angle so our controller will work fine).

So how do you decide exactly where to place the target for this example?  Here is a guide we think may help

1.  Look at the live video feed from the camera
2.  Imagine the path that the robot should follow (try to approximate the distance it needs to avoid running off road etc.)
3.  Place the target as far along this path as it can go so that the robot could head straight to the target without 'running off' the road.

> For example, if we're on a very straight road, we could place it at the horizon.  If we're on a sharp turn, it may need to be placed closer to the robot so it doesn't run out of boundaries.

Assuming our deep learning model works as intended, these labeling guidelines should ensure the following:

1.  The robot can safely travel directly towards the target (without going out of bounds etc.)
2.  The target will continuously progress along our imagined path

What we get, is a 'carrot on a stick' that moves along our desired trajectory.  Deep learning decides where to place the carrot, and JetBot just follows it :)

### Import Libraries

So lets get started by importing all the required libraries for "data collection" purpose. We will mainly use OpenCV to visualize and save image with labels. Libraries such as uuid, datetime are used for image naming. 

In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### Display Live Camera Feed

First, let's initialize and display our camera like we did in the teleoperation notebook. 

We use Camera Class from JetBot to enable CSI MIPI camera. Our neural network takes a 224x224 pixel image as input. We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task). In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [2]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

#display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

### Collect data

The following block of code will display the live image feed, as well as the number of images we've saved.  We store
the target X, Y values by

1. Place the green dot on the target
2. Press 'down' on the DPAD to save

This will store a file in the ``dataset_xy`` folder with files named

``xy_<x value>_<y value>_<uuid>.jpg``

When we train, we load the images and parse the x, y values from the filename

In [3]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

out = widgets.Output()

robot = Robot(driver_board = "dfrobot")
              
motor_speed_ratio = 0.4
time_interval = 0.5

##### Button Function ######

def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()

def step_backward(change):
    robot.backward(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()

def step_left(change):
    robot.left(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()

def step_right(change):
    robot.right(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()

def save_snapshot(change):
    uuid = xy_uuid(x_slider.value, y_slider.value) 
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
        #with out: 
            #print(type(image_widget.value))
            #print(image_path)
        count_widget.value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

Directories not created becasue they already exist
board begin success


In [ ]:
##### Self-defined layout ######

button_layout = widgets.Layout(width='100px', height='80px', align_self='center')

stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])

save_button = widgets.Button(description='save', layout=button_layout)##
count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))


slider_box = widgets.VBox([x_slider, y_slider,count_widget,save_button])
display(image_widget)

In [5]:
##### link buttons to actions #####
save_button.on_click(save_snapshot)
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

###################################

display(widgets.HBox([target_widget,slider_box,controls_box]))
display(out)

Output()

### Next

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training. First, we can call the following terminal command to compress our dataset folder into a single zip file.  

> If you're training on the JetBot itself, you can skip this step!

The ! prefix indicates that we want to run the cell as a shell (or terminal) command.

The -r flag in the zip command below indicates recursive so that we include all nested files, the -q flag indicates quiet so that the zip command doesn't print any output

In [6]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}

You should see a file named road_following_<Date&Time>.zip in the Jupyter Lab file browser. You should download the zip file using the Jupyter Lab file browser by right clicking and selecting Download.